In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import sys
import os
from sklearn.externals import joblib
print('Imported all libraries')

In [ ]:
# band = sys.argv[1] # take in argument 1 (band) if running on terminal
band = 'Red'
# band is one of the following 
# ['Red','Green','Blue','Red_Green','Red_Blue','Green_Blue','Red_Green_Blue']
# ['Hue','Saturation','Value','Hue_Saturation','Hue_Value','Saturation_Value','Hue_Saturation_Value']
number_of_bands = '1'
# number_of_bands = sys.argv[2] # take in argument 2 (number of bands) if running on terminal
# number_of_bands is 1,2 or 3
print(band,type(band))
print(number_of_bands,type(number_of_bands))

# Save directories

In [ ]:
save_path = '/home/josue/Codes/FFNN/Outputs/'+band+'/'
cm_path = save_path+'confusion_matrix/'
weight_path = save_path+'weights/'
bias_path = save_path+'bisases/'
validation_path = save_path+'validation/'
miss_path = save_path+'miss/'
summary_path = save_path+'summary/'

## Checking for existing directory

In [ ]:
dir1 = os.path.isdir(save_path)

if dir1 == False:
    print('Creating Saving Directory')
    os.mkdir(save_path)
    os.mkdir(cm_path)
    os.mkdir(weight_path)
    os.mkdir(bias_path)
    os.mkdir(validation_path)
    os.mkdir(miss_path)
    os.mkdir(summary_path)
    print('Created {}'.format(save_path))
    print('Created {}'.format(cm_path))
    print('Created {}'.format(weight_path))
    print('Created {}'.format(bias_path))
    print('Created {}'.format(validation_path))
    print('Created {}'.format(miss_path))
    print('Created {}'.format(summary_path))
else:
    print('Directory Already Exist')

# Load Images and Labels

In [ ]:
load_path = '/scratch/josue/Data/Data_Frames/Scaled/'+band+'/' 
Training_images = joblib.load(load_path+band+'_Band_Training_Images').values
Training_labels = joblib.load(load_path+band+'_Band_Training_Labels').values
print(Training_images.shape)
print(Training_labels.shape)


Test_images = joblib.load(load_path+band+'_Band_Test_Images').values
Test_labels = joblib.load(load_path+band+'_Band_Test_Labels').values
print(Test_images.shape)
print(Test_labels.shape)


print("Loaded:"+load_path+band+'_Band_Training_Images')
print("Loaded:"+load_path+band+'_Band_Training_Labels')
print("Loaded:"+load_path+band+'_Band_Test_Images')
print("Loaded:"+load_path+band+'_Band_Test_Labels')

# Get image class

In [ ]:
data_test_cls = np.argmax(Test_labels, axis=1)

# Data Parameters


## The size of the original image

In [ ]:
if number_of_bands == '1':
    img_shape = (72,128,1)
if number_of_bands == '2':
    img_shape = (72,128,2)
#     print("Image shape is {}".format(img_shape))
if number_of_bands == '3':
    img_shape = (72,128,3)

## Flatten Image demensions 

In [ ]:
img_size_flat = img_shape[0]*img_shape[1]*img_shape[2]
print("flat image length is {}".format(img_size_flat))
# Number of classes, one class for each category (dust,non_dust)
num_classes = 2
print("Number of classifications {}".format(num_classes))

# TensorFlow Graph


In [ ]:
x = tf.placeholder(tf.float32, [None, img_size_flat]) # placeholder for input data
y_true = tf.placeholder(tf.float32, [None, num_classes]) # placeholder for input labels 
y_true_cls = tf.placeholder(tf.int64, [None]) # placeholder for true classes

# Variables to Optimize


In [ ]:
weights = tf.Variable(tf.zeros([img_size_flat, num_classes])) # initializes weights as zeros 
biases = tf.Variable(tf.zeros([num_classes]))  # initializes biases as zeros 

# Model (Multinomial Classification)


In [ ]:
logits = tf.matmul(x, weights) + biases  # sets up linear model
y_pred = tf.nn.softmax(logits) # activation function use softmax for multiple classification use sigmoid for binary
y_pred_cls = tf.argmax(y_pred, axis=1) # argmax shows the index that has the highest output

# Cost Functions


In [ ]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,labels=y_true) # computes distance from logits and labels
cost = tf.reduce_mean(cross_entropy) # reduces the cost or loss also knows as backpropagation

# Optimization Method

In [ ]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=alpha).minimize(cost) # uses gradient desent 
correct_prediction = tf.equal(y_pred_cls, y_true_cls) # checks for correct predictions
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)) # calculates accuracy



# Run TensorFlow 


In [ ]:
session = tf.Session() 
session.run(tf.global_variables_initializer()) 

# Optimizing Functions


optimize gets batches of images and labels then runs tensor flow session to optimeze model

In [ ]:
def optimize(num_iterations,cnt):
    for i in range(num_iterations):
        batch_index = np.random.choice(len(Training_images), size=batch_size) # segment the data
        x_batch = Training_images[batch_index] # get batch of images  
        y_true_batch = Training_labels[batch_index] # get batch of labels 
        session.run(optimizer, feed_dict={x: x_batch, y_true: y_true_batch}) # run the optimization 
        
        cm = Confusion_matrix(); # generate confusion matrix
        w = session.run(weights) # generate weights
        b = session.run(biases) # generate bias
        
        cm_file = cm_path+'/cm_'+'{0:0>4}'.format(cnt) # set name for confusion matrix file
        weight_file = weight_path+'Weights_'+'{0:0>4}'.format(cnt) # set name for weights file
        bias_file =   bias_path+'Bias_'+'{0:0>4}'.format(cnt) # set name for bias file
        
        np.save(cm_file, cm) # save confusion matrix
        np.save(weight_file, w) # save weights
        np.save(bias_file, b) # save biases
        
        cnt += 1
    return w,b,cm

print_accuracy runs tensor flow session and calculates accuracy

In [ ]:
def print_accuracy():
    # Use TensorFlow to compute the accuracy.
    acc = session.run(accuracy, feed_dict=feed_dict_test)
    
    # Print the accuracy.
    print("Accuracy on test-set: {0:.1%}".format(acc))

Confusion_matrix runs tensor flow session and compares predicted class vs true class and generates confusion matrix

In [ ]:
def Confusion_matrix():
    # Get the true classifications for the test-set.
    cls_true = data_test_cls
    
    # Get the predicted classifications for the test-set.
    cls_pred = session.run(y_pred_cls, feed_dict=feed_dict_test)

    # Get the confusion matrix using sklearn.
    cm = confusion_matrix(y_true=cls_true,
                          y_pred=cls_pred)
    return cm

# Performance before any optimization

## feed in data to placeholders

In [ ]:
feed_dict_test = {x: Test_images,
                  y_true: Test_labels,
                  y_true_cls: data_test_cls}

In [ ]:
print_accuracy() # print accuracy at intial state of model 

# Optimizing 

In [ ]:
cnt = 0 # counter 
acc = 0 # initial accuracy 
batch_size = 10 # batch size
alpha = 0.5 # learning rate

while acc < .95: # break if accuracy reaches 95
    if cnt == 500: # break if counter reaches 500 iterations
        break
    w,b,cm = optimize(1,cnt)  # optimeze model save weights, bias, confusion matrix
    acc = session.run(accuracy, feed_dict=feed_dict_test) # calculate accuracy 
    cnt += 1 # increase counter by 1
print('Number of iterations: {}'.format(cnt))
print('Final Testing Accuracy: {}'.format(acc))
print('Confusion matrix for testing set: {}'.format(cm))

# Set up tensorflow to validate images

In [ ]:
Validation_images = joblib.load(load_path+band+'_Band_Validation_Images').values # load validation image
Validation_labels = joblib.load(load_path+band+'_Band_Validation_Labels').values # load validation labels

print("Loaded:"+load_path+band+'_Band_Validation_Images')
print("Loaded:"+load_path+band+'_Band_Validation_Labels')

In [ ]:
Weights = os.listdir(weight_path) # list of all models weights 
Bias = os.listdir(bias_path) # list of all models bias 

Accuracy = [] # empty array for accuracy 
Recall = [] # empty array for recall 
Precision = [] # empty array for precision 

In [ ]:
for i in range(len(Weights)): # loop through all weights
    x_validation = tf.placeholder(tf.float32, [None, Validation_images.shape[1]]) # placeholder for new input data
    weights = np.load(weight_path+'Weights_'+'{0:0>4}.npy'.format(i)) # load weights
    biases = np.load(bias_path+'Bias_'+'{0:0>4}.npy'.format(i)) # load biases

    logits = tf.matmul(x_validation, weights) + biases  # sets up linear model
    y_pred = tf.nn.softmax(logits) # activation function use softmax for multiple classification use sigmoid for binary
    y_pred_cls = tf.argmax(y_pred, axis=1) # argmax shows the index that has the highest output

    session = tf.Session()  # create tensor flow session
    session.run(tf.global_variables_initializer()) # initialize session
    feed_dict_val = {x_validation: Validation_images} # feed in validation images

    prediction = session.run(y_pred_cls, feed_dict=feed_dict_val) # run session
    data_validation_cls = np.argmax(Validation_labels, axis=1) # classify preditions

    cm = confusion_matrix(y_true=data_validation_cls,y_pred=prediction) # obtain confusion matrix
    validation_file = validation_path+'/cm_'+'{0:0>4}'.format(i) # validation file path
    np.save(validation_file, cm) # save confusion matrix 

    temp = np.reshape(cm,4).astype(float) # reshape confusion matrix
    accuracy = (temp[0]+temp[3])/np.sum(temp) # calculate accuracy 
    recall = float((temp[-1])/(temp[-1]+temp[-2]))  # calculate recall
    precision = float((temp[-1])/(temp[-1]+temp[-3])) # calculate precision 
    Accuracy.append(accuracy) # append accuracy 
    Recall.append(recall) # append recall
    Precision.append(precision) # append precision
    

    for j in range(len(prediction)): # loop through all predictions 
        if prediction[j] != data_validation_cls[j]: # check if prediction matches label
            # create text file of misclassified frames
            with open(miss_path+'Misclassification_Index_'+'{0:0>4}'.format(i)+'.txt', 'a') as the_file:
                the_file.write('wrong classification at index {}\n'.format(j))
    print('Completed Validation for {}'.format(i))

np.save(summary_path+'accuracy', Accuracy) # save accuracy 
np.save(summary_path+'recall', Recall) # save recall
np.save(summary_path+'precision', Precision) # save precision 